# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1451658135 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Users\hvrigazov\AppData\Local\Dato\Dato Launcher\lib\site-packages\certifi\cacert.pem
1451658135 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to hrivri@gmail.com and will expire on October 03, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-2124 - Server binary: C:\Users\hvrigazov\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\HVRIGA~1\AppData\Local\Temp\graphlab_server_1451658135.log.0
[INFO] GraphLab Server Version: 1.7.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.402230     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.449107     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [6]:
for weight in model_all.coefficients:
    if weight['value'] > 0:
        print weight

{'index': None, 'name': '(intercept)', 'value': 274873.0559504957}
{'index': None, 'name': 'bathrooms', 'value': 8468.531086910072}
{'index': None, 'name': 'sqft_living', 'value': 24.420720982445214}
{'index': None, 'name': 'sqft_living_sqrt', 'value': 350.0605533860648}
{'index': None, 'name': 'grade', 'value': 842.0680348976282}
{'index': None, 'name': 'sqft_above', 'value': 20.024722417091304}


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
best_l1_penalty = 0
best_RSS_validation_l1_penalty = float('inf')
best_l1_penalty_RSS_test = float('inf')
best_l1_coefficients = list()

for l1_penalty in np.logspace(1, 7, num=13):
    current_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty, verbose = False)
    
    RSS_current_model = sum((current_model.predict(validation) - validation['price']) ** 2)
    if RSS_current_model < best_RSS_validation_l1_penalty:
        best_RSS_validation_l1_penalty = RSS_current_model
        best_l1_penalty_RSS_test = sum((current_model.predict(testing) - testing['price']) ** 2)
        best_l1_penalty = l1_penalty
        best_l1_coefficients = current_model.coefficients
        
print "l1 penalty: " + str(best_l1_penalty)
print "best RSS test:" + str(best_l1_penalty_RSS_test)
print "Coefficients: "

count = 0
for weight in best_l1_coefficients:
    if weight['value'] > 0:
        print weight
        count += 1
        
print count

l1 penalty: 10.0
best RSS test:1.56983602382e+14
Coefficients: 
{'index': None, 'name': '(intercept)', 'value': 18993.427212770577}
{'index': None, 'name': 'bedrooms', 'value': 7936.967679031306}
{'index': None, 'name': 'bedrooms_square', 'value': 936.993368193299}
{'index': None, 'name': 'bathrooms', 'value': 25409.58893412067}
{'index': None, 'name': 'sqft_living', 'value': 39.11513637970762}
{'index': None, 'name': 'sqft_living_sqrt', 'value': 1124.6502128077207}
{'index': None, 'name': 'sqft_lot', 'value': 0.0034836182229897442}
{'index': None, 'name': 'sqft_lot_sqrt', 'value': 148.2583910114082}
{'index': None, 'name': 'floors', 'value': 21204.33546695013}
{'index': None, 'name': 'floors_square', 'value': 12915.524336072436}
{'index': None, 'name': 'waterfront', 'value': 601905.5945452718}
{'index': None, 'name': 'view', 'value': 93312.85731187189}
{'index': None, 'name': 'condition', 'value': 6609.035712447216}
{'index': None, 'name': 'grade', 'value': 6206.939991880551}
{'index'

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [9]:
print count

18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [27]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [29]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [30]:
non_zeros_count = {}
for l1_penalty in l1_penalty_values:
    current_model = graphlab.linear_regression.create(dataset=training, target='price', features=all_features, \
                                              l1_penalty=l1_penalty, l2_penalty=0, validation_set = None, verbose=False)
    non_zeros_count[current_model['coefficients']['value'].nnz()] = l1_penalty
    
print non_zeros_count

{1: 10000000000.0, 3: 6158482110.6602545, 5: 4832930238.5717525, 6: 3792690190.7322536, 10: 2976351441.6313128, 12: 2335721469.0901213, 13: 1832980710.8324375, 15: 1438449888.2876658, 16: 885866790.41008317, 17: 695192796.17755914, 18: 206913808.11147901}


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [32]:
l1_penalty_min = non_zeros_count[max([x for x in non_zeros_count if x < max_nonzeros])]
l1_penalty_max = non_zeros_count[min([x for x in non_zeros_count if x > max_nonzeros])]

print l1_penalty_min
print l1_penalty_max

3792690190.73
2976351441.63


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [33]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [38]:
lowest_RSS = float('inf')
best_l1_penalty = float('inf')
best_coefficients = {}
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(dataset=training, target='price', features=all_features, \
                                              l1_penalty=l1_penalty, l2_penalty=0, validation_set = None, verbose=False)
    RSS = sum((model.predict(validation) - validation['price']) ** 2)
    sparsity = model['coefficients']['value'].nnz()
    print sparsity
    if sparsity == max_nonzeros and RSS < lowest_RSS:
        print model['coefficients']['value'].nnz()
        best_coefficients = model['coefficients']
        lowest_RSS = RSS
        best_l1_penalty = l1_penalty
        
print lowest_RSS

for coefficient in best_coefficients:
    if not coefficient['value'] == 0:
        print coefficient

6
6
6
6
6
7
7
7
7
7
7
7
7
8
8
8
10
10
10
10
10
10
10
10
1.04693748875e+15
{'index': None, 'name': '(intercept)', 'value': 222253.19254432785}
{'index': None, 'name': 'bedrooms', 'value': 661.7227177822587}
{'index': None, 'name': 'bathrooms', 'value': 15873.957259267981}
{'index': None, 'name': 'sqft_living', 'value': 32.41022145125964}
{'index': None, 'name': 'sqft_living_sqrt', 'value': 690.1147733133256}
{'index': None, 'name': 'grade', 'value': 2899.4202697498786}
{'index': None, 'name': 'sqft_above', 'value': 30.011575302201045}


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?